# 1. Обробка сирих даних

Імпорт необхідних бібліотек

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import fnmatch


Зчитування CSV файлів датасету

In [33]:
# OList eCommerce Public Data Set
olistDir = "./dataset/"

OL_customers = pd.read_csv(olistDir+"olist_customers_dataset.csv")
OL_geo = pd.read_csv(olistDir+"olist_geolocation_dataset.csv")
OL_orders = pd.read_csv(olistDir+"olist_orders_dataset.csv")
OL_orders_items = pd.read_csv(olistDir+"olist_order_items_dataset.csv")
OL_orders_payments = pd.read_csv(olistDir+"olist_order_payments_dataset.csv")
OL_orders_reviews = pd.read_csv(olistDir+"olist_order_reviews_dataset.csv")
OL_products = pd.read_csv(olistDir+"olist_products_dataset.csv")
OL_sellers = pd.read_csv(olistDir+"olist_sellers_dataset.csv")
OL_products_translations = pd.read_csv(
    olistDir+"product_category_name_translation.csv")


OL_customers = OL_customers.dropna(thresh=len(OL_customers.columns)-1)
OL_geo = OL_geo.dropna(thresh=len(OL_geo.columns)-1)
OL_orders = OL_orders.dropna(thresh=len(OL_orders.columns)-1)
OL_orders_items = OL_orders_items.dropna(thresh=len(OL_orders_items.columns)-1)
OL_orders_payments = OL_orders_payments.dropna(
    thresh=len(OL_orders_payments.columns)-1)
OL_orders_reviews = OL_orders_reviews.dropna(
    thresh=len(OL_orders_reviews.columns)-1)
OL_products = OL_products.dropna(thresh=len(OL_products.columns)-1)
OL_sellers = OL_sellers.dropna(thresh=len(OL_sellers.columns)-1)
OL_products_translations = OL_products_translations.dropna(
    thresh=len(OL_products_translations.columns)-1)


Об'єднання кастомерів

In [34]:
dropGeo = OL_geo.drop_duplicates("geolocation_zip_code_prefix")
dropGeo = dropGeo[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]

mergedOLCustomers = OL_customers.merge(dropGeo, how='left',left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix")
mergedOLCustomers = mergedOLCustomers.rename(columns={'geolocation_lat':'customer_latitude','geolocation_lng':'customer_longitude'})
mergedOLCustomers = mergedOLCustomers.drop("geolocation_zip_code_prefix",axis=1)

Об'єднання айтемів

In [35]:
productsSub = OL_products[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]
productsSub.insert(len(productsSub.columns),'product_volume_cm3', OL_products['product_length_cm']*OL_products['product_height_cm']*OL_products['product_width_cm'])
productsSub.insert(len(productsSub.columns),'product_density_g_per_cm3',OL_products['product_weight_g']/productsSub['product_volume_cm3'])

mergedOLProducts = productsSub.merge(OL_products_translations, how='left', on="product_category_name")
mergedOLProduct = mergedOLProducts.drop("product_category_name",axis=1)
mergedOLItems = OL_orders_items.merge(mergedOLProducts, how='left', on="product_id")
mergedOLItems = mergedOLItems.drop("product_category_name",axis=1)

Об'єднання продавців

In [36]:
mergedSellers = OL_sellers.merge(dropGeo, how='left', left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix")
mergedSellers = mergedSellers.drop("geolocation_zip_code_prefix",axis=1)
mergedSellers = mergedSellers.rename(columns={'geolocation_lat':'seller_latitude','geolocation_lng':'seller_longitude'})
mergedOLItems = mergedOLItems.merge(mergedSellers, how='left', on="seller_id")

Об'єднання всіх даних

In [37]:
ReviewsSub = OL_orders_reviews[['review_id', 'order_id', 'review_score', 'review_creation_date', 'review_answer_timestamp']]

OL_Data = OL_orders.merge(OL_orders_payments, how='left', on='order_id')
OL_Data = OL_Data.merge(ReviewsSub, how='left', on='order_id')
OL_Data = OL_Data.merge(mergedOLItems, how='left', on='order_id')
OL_Data = OL_Data.merge(mergedOLCustomers, how='left', on='customer_id')

Додаткові методи

In [38]:
import itertools
def PrintDataFrameSummary(frame):
    metaData = list()
    metaData.append(list(["Name:","DType:","Non-Null:","Unique:"]))
    for col in list(frame.columns):
        metaData.append( list(["{0}".format(col), 
                              "{0}".format(frame[col].dtype),
                              "{0}".format(frame[col].notna().sum()), 
                              "{0}".format(len(set(frame[frame[col].notna()][col])))]))
    
    formatString = ""
    
    for i in range(len(metaData[0])):
        maxLength = len(max([l[i] for l in metaData], key=len))
        formatString += "{"+":<{0}".format(maxLength+8) +"}"
        
    for i in range(len(metaData)):
        
        print(formatString.format(*metaData[i]))
    
    
        
    print()
    print("Initial Length {0}".format(len(frame)))
    print("Dropped NA Length {0}".format(len(frame.dropna())))
    return frame.head()

Перевірка даних

In [39]:
PrintDataFrameSummary(OL_Data)

Name:                                DType:         Non-Null:        Unique:        
order_id                             object         116554           97659          
customer_id                          object         116554           97659          
order_status                         object         116554           3              
order_purchase_timestamp             object         116554           97115          
order_approved_at                    object         116539           89287          
order_delivered_carrier_date         object         116553           81018          
order_delivered_customer_date        object         115228           95664          
order_estimated_delivery_date        object         116554           446            
payment_sequential                   float64        116551           29             
payment_type                         object         116551           4              
payment_installments                 float64        116551       

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,seller_city,seller_state,seller_latitude,seller_longitude,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_latitude,customer_longitude
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,credit_card,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3.0,voucher,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2.0,voucher,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,boleto,...,belo horizonte,SP,-19.810119,-43.984727,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.169860,-44.988369
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,credit_card,...,guariba,SP,-21.362358,-48.232976,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.746337,-48.514624


In [40]:
OL_Data.dtypes

order_id                          object
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
payment_sequential               float64
payment_type                      object
payment_installments             float64
payment_value                    float64
review_id                         object
review_score                     float64
review_creation_date              object
review_answer_timestamp           object
order_item_id                    float64
product_id                        object
seller_id                         object
shipping_limit_date               object
price                            float64
freight_value                    float64
product_weight_g                 float64
product_length_cm                float64
product_height_c

Конвертація дати

In [41]:
for dateCol in ['order_purchase_timestamp',
                'order_approved_at',
                'order_delivered_carrier_date',
                'order_delivered_customer_date', 
                'order_estimated_delivery_date',
                'review_creation_date',
                'review_answer_timestamp',
                'shipping_limit_date'
               ]:
    OL_Data[dateCol] = pd.to_datetime(OL_Data[dateCol])

Статуси замовлень

In [42]:
OL_Data['order_status'].value_counts()

delivered    115228
shipped        1246
canceled         80
Name: order_status, dtype: int64

In [43]:
OL_Data['payment_type'].value_counts()

credit_card    86005
boleto         22629
voucher         6237
debit_card      1680
Name: payment_type, dtype: int64

Генерація одного датасету

In [44]:
#Total Price and Total Items in Order
itemSub = OL_Data[['order_id', 'order_item_id','price','product_id']]
itemSub = itemSub.drop_duplicates()
itemGroup = itemSub.groupby('order_id', as_index=False).sum()
itemGroup['number_of_items'] = itemGroup['order_item_id'].map({0:0,
                                                                1:1,
                                                                3:2,
                                                                6:3,
                                                                10:4,
                                                                15:5,
                                                                21:6,
                                                                28:7,
                                                                36:8,
                                                                45:9,
                                                                55:10,
                                                                66:11,
                                                                78:12,
                                                                91:13,
                                                                105:14,
                                                                120:15,
                                                                210:20,
                                                                231:21})
itemGroup = itemGroup.drop(['order_item_id'],axis=1)
itemGroup = itemGroup.rename(columns={"price":"Total_price"})
print("itemGroup")
print(itemGroup.head())


#Median Payments, useful for installment plans
paymentSub = OL_Data[['order_id',
                      'payment_value',
                      'payment_type',
                      'freight_value',
                     'payment_sequential']]
paymentSub = paymentSub.drop_duplicates()
payments = paymentSub[['order_id','payment_value','freight_value']]
paymentsMed = payments[['order_id','payment_value']].groupby('order_id', as_index=False).median()
paymentsMed = paymentsMed.rename(columns ={"payment_value":"Median_payments"})
print()
print("paymentsMed")
print(paymentsMed.head())

#Total Payments, Total Frieght Cost Value
paymentsTot = payments.groupby('order_id', as_index=False).sum()
paymentsTot = paymentsTot.rename(columns = {"payment_value":"Total_payment","freight_value":"Total_Freight"})
print()
print("paymentsTot")
print(paymentsTot.head())

#Total Weight and Volume of Order
ordersSize = OL_Data[['order_id', 
                        'order_item_id',
                        'price',
                        'product_id',
                        'product_weight_g',
                        'product_volume_cm3']]
ordersSize = ordersSize.drop_duplicates()
ordersSize = ordersSize[['order_id',
                         'product_weight_g',
                        'product_volume_cm3']].groupby('order_id', as_index=False).sum()
ordersSize = ordersSize.rename(columns = {'product_weight_g':'order_weight_g',
                                         'product_volume_cm3':'order_volume_cm3'})
print()
print("ordersSize")
print(ordersSize.head())

#Payment Type
paymentsType = paymentSub[['order_id','payment_sequential','payment_type']]
paymentsType = paymentsType.sort_values(by='payment_sequential')[['order_id', 'payment_type']]
paymentsType = paymentsType.groupby('order_id', as_index=False).first()
print()
print("paymentsType")
print(paymentsType.head())


#Merge to Single DataFrame.
#An order may still have multiple sellers
NonGroupedData = OL_Data[['order_id',  
                          'order_status', 
                          'order_purchase_timestamp',
                          'order_approved_at', 
                          'order_delivered_carrier_date',
                          'order_delivered_customer_date', 
                          'order_estimated_delivery_date',
                          'shipping_limit_date',
                          'payment_installments',
                          'product_category_name_english', 
                          'seller_id', 
                          'seller_zip_code_prefix',
                          'seller_city', 
                          'seller_state', 
                          'seller_latitude', 
                          'seller_longitude',
                          'customer_id',
                          'customer_unique_id', 
                          'customer_zip_code_prefix', 
                          'customer_city',
                          'customer_state', 
                          'customer_latitude', 
                          'customer_longitude',
                          'review_id', 
                          'review_score', 
                          'review_creation_date',
                          'review_answer_timestamp']]
NonGroupedData= NonGroupedData.drop_duplicates()
OrderData = NonGroupedData.merge(itemGroup,on='order_id')
OrderData = OrderData.merge(paymentsMed, on='order_id')
OrderData = OrderData.merge(paymentsTot, on='order_id')
OrderData = OrderData.merge(ordersSize, on='order_id')
OrderData = OrderData.merge(paymentsType, on='order_id')


#Generate Delivery Features
OrderData['Purchase_To_Delivery_Days'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_purchase_timestamp']]
OrderData['Approved_To_Delivery_Days'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_approved_at']]
OrderData['Diff_Est_Delivery_vs_Actual'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_estimated_delivery_date']]

OrderData.to_csv("./dataset/OL_Data_Order_Grouped.csv")

C:\Users\Vlad\AppData\Local\Temp\ipykernel_18812\1632512777.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  itemGroup = itemSub.groupby('order_id', as_index=False).sum()


itemGroup
                           order_id  Total_price  number_of_items
0  00010242fe8c5a6d1ba2dd792cb16214        58.90                1
1  00018f77f2f0320c557190d7a144bdd3       239.90                1
2  000229ec398224ef6ca0657da4fc703e       199.00                1
3  00024acbcdf0a6daa1e931b038114c75        12.99                1
4  00042b26cf59d7ce69dfabb4e55b4fd9       199.90                1

paymentsMed
                           order_id  Median_payments
0  00010242fe8c5a6d1ba2dd792cb16214            72.19
1  00018f77f2f0320c557190d7a144bdd3           259.83
2  000229ec398224ef6ca0657da4fc703e           216.87
3  00024acbcdf0a6daa1e931b038114c75            25.78
4  00042b26cf59d7ce69dfabb4e55b4fd9           218.04

paymentsTot
                           order_id  Total_payment  Total_Freight
0  00010242fe8c5a6d1ba2dd792cb16214          72.19          13.29
1  00018f77f2f0320c557190d7a144bdd3         259.83          19.93
2  000229ec398224ef6ca0657da4fc703e         216.87  

# 2. Recomendation

In [45]:
# import libraries
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from IPython.display import clear_output
import copy

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#getRecommendataionsFromCategoryGroupDict
#  Take a list of orders, a unique customer id in the orders and a Category DataFrame (columns represent 
#  categories of the Orders, and each row is a Group that sums to 100% with this percentage spread out 
#  among the columns and every row is unique) and return a dictionary of product ids as keys and the cosine
#  simularity between this users orders and a row of the Category DataFrame.  
#
#  Only products that the customer has not bought before will appear in the returned dictionary, and products
#  that are repeated will be listed once with its highest simularity score.  In addition, the user passes in
#  a minimum simularity score, and any Group whose simularity with this user is below this threshold will not
#  appear in the returned recommendations
#
#  orders - a list of orders from the OL_Data dataset, either the whole thing or simplified columns
#  custUniqueId - the ID of the customer to create recommendations for
#  productColName - name of the column that contains the unique product identifier
#  customerColName - name of the column that contains the unique customer identifier
#  categoryGroupsDF - Category DataFrame as described above, columns are categories, rows are Groups
#  categoryGroupsDF_DropCols - Columns in categoryGroupsDF to ignore
#  categoryGroupsDF_CatColumnName - the column name in the orders that contains the categories in the columns
#                                   of categoryGroupsDF
#  categoryGroupsDF_groupIdName - the column name of the unique Group (row) identifier in categoryGroupsDF
#  minimumSimularity - value between 0 and 1, any simularity lower than this will be ignored
#  includeRFM (optional) - Boolean to include/exclude the RFM score in recommendation
#  RFM_ColName (optional) - Column for RFM Score, only used if includeRFM is True
def getRecommendataionsFromCategoryGroupDict(orders,
                                             custUniqueId,
                                             
                                             productColName,
                                             customerColName,
                                             
                                             categoryGroupsDF,
                                             categoryGroupsDF_DropCols,
                                             categoryGroupsDF_CatColumnName,
                                             categoryGroupsDF_groupIdName,
                                             
                                             minimumSimularity,
                                             
                                             includeRFM = False,
                                             RFM_ColName = None):

    #Remove ignored columns from CategoryGroupsDF, and set index
    catGroupsDF_Indexed = categoryGroupsDF.drop(columns=categoryGroupsDF_DropCols).set_index(categoryGroupsDF_groupIdName)
    
    #Get all categories in the Group
    all_categories = list(catGroupsDF_Indexed.columns)
    all_categories = [str(x) for x in all_categories]    
    
    #Set up the Product Recommendation Dict, empty dict returned if no user orders, or no simularity high enough
    RecommendationDict = {}    
    RecommendationDict['IdType']=categoryGroupsDF_groupIdName+"_CosSim"
    
    #Create the Row for the customer to use for cosine simularity
    thiscustSubset = orders[orders[customerColName] == custUniqueId]
    thisCustCategories = np.array([len(thiscustSubset[thiscustSubset[categoryGroupsDF_CatColumnName] == x]) for x in all_categories])
    
    #No user orders
    if(sum(thisCustCategories) == 0):
        return RecommendationDict
    
    #Normalize customer's row to percentages in categories, and create DF
    thisCustCategories = 100*thisCustCategories/sum(thisCustCategories)
    thisCustCategories.shape = (1, len(thisCustCategories))
    custRowDF = pd.DataFrame(data=thisCustCategories, columns=all_categories)    

    #Calculate simularity scores, and add to dataframe for sorting against Group ID
    simularity = cosine_similarity(catGroupsDF_Indexed, custRowDF)
    simDF = pd.DataFrame(simularity, columns=['Simularity'])
    simDF[categoryGroupsDF_groupIdName] = catGroupsDF_Indexed.reset_index()[categoryGroupsDF_groupIdName]
    
    #Get orders with product_ids this user has not bought
    thisCustomerProducts = thiscustSubset[productColName].unique()
    newProductOrders = OL_Data_Simp[~OL_Data_Simp[productColName].isin(thisCustomerProducts)]    
    
    #Filter out Group IDs that are not simular enough
    SimilarEnoughDF = simDF[simDF.Simularity>=minimumSimularity].sort_values(by='Simularity', ascending=False) 
    for index, row in SimilarEnoughDF.iterrows():
        
        prob = row['Simularity']
            
        if includeRFM and RFM_ColName != None:
            maxRFM = max(orders[RFM_ColName])
            
            #Get Product Ids, keeping the id with the highest specified RFM
            thisProdRecs = newProductOrders[newProductOrders[categoryGroupsDF_groupIdName]==row[categoryGroupsDF_groupIdName]][[productColName, RFM_ColName]]
            thisProdRecs = thisProdRecs.sort_values(by=RFM_ColName)
            thisProdRecs = thisProdRecs.drop_duplicates(subset=[productColName])
            
            #Go through collected Product Ids, replacing existing if new score is higher
            #or adding new
            for i,row in thisProdRecs.iterrows():
                adjustedProb = prob+row[RFM_ColName]/maxRFM
                
                if(row[productColName] in RecommendationDict.keys()):
                    RecommendationDict[row[productColName]] = max(RecommendationDict[row[productColName]], adjustedProb)
                else:
                    RecommendationDict[row[productColName]] = adjustedProb
        else:
            #Get product_ids of this Group, and add those not already in the Recommendations
            thisProdRecs = newProductOrders[newProductOrders[categoryGroupsDF_groupIdName] == row[categoryGroupsDF_groupIdName]][productColName].unique()

            for prod in thisProdRecs:
                if(prod in RecommendationDict.keys()):
                    continue
                
                RecommendationDict[prod] = prob
    #Return the product recommendations
    return RecommendationDict

In [ ]:
#getRecommendataionsFromSimilarityMatrix
#  Take a list of orders, a unique customer id in the orders, and a similarity matrix with similarity scores
#  between groups.  Also provided are the name of the Group category, which needs to be in the list of orders
#  and the user given needs to be assigned to one.  Using the similarity matrix, recommendations will be given 
#  from Groups in order of their similarity to the customer
#
#  Only products that the customer has not bought before will appear in the returned dictionary, and products
#  that are repeated will be listed once with its highest simularity score.  In addition, the user passes in
#  a minimum simularity score, and any Group whose simularity with this user is below this threshold will not
#  appear in the returned recommendations
#
#  orders - a list of orders from the OL_Data dataset, either the whole thing or simplified columns
#  custUniqueId - the ID of the customer to create recommendations for
#  productColName - name of the column that contains the unique product identifier
#  customerColName - name of the column that contains the unique customer identifier
#  SimilarityMatrix - n x n+1 matrix, with similarity scores of every group to every other, a group ID column
#  GroupIdName - the column name of the unique Group identifier in SimilarityMatrix
#  minimumSimularity - value between 0 and 1, any simularity lower than this will be ignored
#  includeRFM (optional) - Boolean to include/exclude the RFM score in recommendation
#  RFM_ColName (optional) - Column for RFM Score, only used if includeRFM is True
def getRecommendataionsFromSimilarityMatrix(orders,
                                             custUniqueId,
                                             productColName,
                                             customerColName,
                                             
                                             SimilarityMatrix,
                                             GroupIdName,
                                             
                                             minimumSimularity,
                                            
                                             includeRFM = False,
                                             RFM_ColName = None):

    
    GroupId = orders[orders[customerColName]==custUniqueId][GroupIdName].iloc[0]
    
    #Set up the Product Recommendation Dict, empty dict returned if no user orders, or no simularity high enough
    RecommendationDict = {}    
    RecommendationDict['IdType']=GroupIdName+"_CosSim"
    
    #Get Similarity Scores for this Group
    if(len(SimilarityMatrix[SimilarityMatrix[GroupIdName]==GroupId]) == 0 ):
        return RecommendationDict
    
    #Get the Groups with similarity above threshold
    GroupId_Str = str(int(GroupId))
    simScores = SimilarityMatrix[[GroupIdName,GroupId_Str]]
    simScores = simScores[simScores[GroupId_Str]>= minimumSimularity].sort_values(by=GroupId_Str)
    
    #Get orders with product_ids this user has not bought
    thiscustSubset = orders[orders[customerColName] == custUniqueId]
    thisCustomerProducts = thiscustSubset[productColName].unique()
    newProductOrders = OL_Data_Simp[~OL_Data_Simp[productColName].isin(thisCustomerProducts)]    
    
    for index, row in simScores.iterrows():
        
        prob = row[GroupId_Str]
            
        if includeRFM and RFM_ColName != None:
            maxRFM = max(orders[RFM_ColName])
            
            #Get Product Ids, keeping the id with the highest specified RFM
            thisProdRecs = newProductOrders[newProductOrders[GroupIdName]==row[GroupIdName]][[productColName, RFM_ColName]]
            thisProdRecs = thisProdRecs.sort_values(by=RFM_ColName)
            thisProdRecs = thisProdRecs.drop_duplicates(subset=[productColName])
            
            #Go through collected Product Ids, replacing existing if new score is higher
            #or adding new
            for i,row in thisProdRecs.iterrows():
                adjustedProb = prob+row[RFM_ColName]/maxRFM
                
                if(row[productColName] in RecommendationDict.keys()):
                    RecommendationDict[row[productColName]] = max(RecommendationDict[row[productColName]], adjustedProb)
                else:
                    RecommendationDict[row[productColName]] = adjustedProb
        else:
            #Get product_ids of this Group, and add those not already in the Recommendations
            thisProdRecs = newProductOrders[newProductOrders[GroupIdName] == row[GroupIdName]][productColName].unique()

            for prod in thisProdRecs:
                if(prod in RecommendationDict.keys()):
                    continue
                
                RecommendationDict[prod] = prob
    #Return the product recommendations
    return RecommendationDict

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples
from IPython.display import clear_output
import copy


#getRecommendataionsFromAglomCluster
#  Take a list of orders, a unique customer id in the orders and a Category DataFrame (columns represent 
#  categories of the Orders, and each row is a Group that sums to 100% with this percentage spread out 
#  among the columns and every row is unique) and return a dictionary of product ids as keys and the silouette
#  score of the Groups in the same cluster as the customer
#
#  Only products that the customer has not bought before will appear in the returned dictionary, and products
#  that are repeated will be listed once with its highest silouette score.  In addition, the user passes in
#  a minimum simularity score, and any Group whose simularity with this user is below this threshold will not
#  appear in the returned recommendations
#
#  orders - a list of orders from the OL_Data dataset, either the whole thing or simplified columns
#  custUniqueId - the ID of the customer to create recommendations for
#  productColName - name of the column that contains the unique product identifier
#  customerColName - name of the column that contains the unique customer identifier
#  categoryGroupsDF - Category DataFrame as described above, columns are categories, rows are Groups
#  categoryGroupsDF_DropCols - Columns in categoryGroupsDF to ignore
#  categoryGroupsDF_CatColumnName - the column name in the orders that contains the categories in the columns
#                                   of categoryGroupsDF
#  categoryGroupsDF_groupIdName - the column name of the unique Group (row) identifier in categoryGroupsDF
#  minimumSimularity - value between 0 and 1, any simularity lower than this will be ignored
#  includeRFM (optional) - Boolean to include/exclude the RFM score in recommendation
#  RFM_ColName (optional) - Column for RFM Score, only used if includeRFM is True
def getRecommendataionsFromAglomCluster(orders,
                                        custUniqueId,
                                        
                                         productColName,
                                         customerColName,

                                         categoryGroupsDF,
                                         categoryGroupsDF_DropCols,
                                         categoryGroupsDF_CatColumnName,
                                         categoryGroupsDF_groupIdName,

                                         minimumSimularity,
                                         includeRFM = False,
                                         RFM_ColName = None):
    
    #Remove ignored columns from CategoryGroupsDF, and set index
    catGroupsDF_Indexed = categoryGroupsDF.drop(columns=categoryGroupsDF_DropCols).set_index(categoryGroupsDF_groupIdName).sort_values(by=categoryGroupsDF_groupIdName)
    
    #Get all categories in the Group
    all_categories = list(catGroupsDF_Indexed.columns)
    all_categories = [str(x) for x in all_categories]    
    
    #Set up the Product Recommendation Dict, empty dict returned if no user orders, or no simularity high enough
    RecommendationDict = {}    
    RecommendationDict['IdType']=categoryGroupsDF_groupIdName+"_Aglom"
    
    #Get the Row for the customer to use Agglomerative Clustering
    thiscustSubset = orders[orders[customerColName] == custUniqueId]
    thisCustCategories = np.array([len(thiscustSubset[thiscustSubset[categoryGroupsDF_CatColumnName] == x]) for x in all_categories])
    
    #No user orders
    if(sum(thisCustCategories) ==0):
        return RecommendationDict
    
    #Normalize customer's row to percentages in categories, and create DF
    thisCustCategories = 100*thisCustCategories/sum(thisCustCategories)
    thisCustCategories.shape = (1, len(thisCustCategories))
    custRowDF = pd.DataFrame(data=thisCustCategories, columns=all_categories)    
    

    #Add customer to Cat Groups for clustering,
    allDF = pd.concat([catGroupsDF_Indexed,custRowDF], ignore_index=True)
    model = AgglomerativeClustering(n_clusters=30, linkage="average").fit(allDF)
    #Get clusters for customer and others
    clusterAssignments = model.labels_
    cusCluster = clusterAssignments[-1]
    otherClusters = clusterAssignments[:-1]
    #Get silouette scores for customer and others
    scores = silhouette_samples(allDF,clusterAssignments, metric ='cosine')
    cusScore = scores[-1]
    otherScores = scores[:-1]
    
    #Add cluster assignments and silouette to Category DF
    catGroupsDF_Indexed['assignedTo'] = otherClusters
    catGroupsDF_Indexed['s_score'] = otherScores
    catGroupsDF_Indexed = catGroupsDF_Indexed.reset_index()
    
    #Get groups in same cluster of user and filter out sillouette score lower than threshold
    group = catGroupsDF_Indexed[catGroupsDF_Indexed['assignedTo']==cusCluster]
    group = group[group['s_score']>=minimumSimularity].sort_values(by='s_score')
    
    #Get orders with products the user has not bought
    thisCustomerProducts = thiscustSubset[productColName].unique()
    newProductOrders = OL_Data_Simp[~OL_Data_Simp[productColName].isin(thisCustomerProducts)]
    
    
    for i,row in group.iterrows():
        prob = row['s_score']
        
        if includeRFM and RFM_ColName != None:
            maxRFM = max(orders[RFM_ColName])
            
            #Get Product Ids, keeping the id with the highest specified RFM
            thisProdRecs = newProductOrders[newProductOrders[categoryGroupsDF_groupIdName] == row[categoryGroupsDF_groupIdName]][[productColName, RFM_ColName]]
            thisProdRecs = thisProdRecs.sort_values(by=RFM_ColName)
            thisProdRecs = thisProdRecs.drop_duplicates(subset=[productColName])
            
            #Go through collected Product Ids, replacing existing if new score is higher
            #or adding new
            for i,row in thisProdRecs.iterrows():
                adjustedProb = prob+row[RFM_ColName]/maxRFM
                
                if(row[productColName] in RecommendationDict.keys()):
                    RecommendationDict[row[productColName]] = max(RecommendationDict[row[productColName]], adjustedProb)
                else:
                    RecommendationDict[row[productColName]] = adjustedProb
        else:
            #Get product_ids of this Group, and add those not already in the Recommendations
            thisProdRecs = newProductOrders[newProductOrders[categoryGroupsDF_groupIdName] == row[categoryGroupsDF_groupIdName]][productColName].unique()

            for prod in thisProdRecs:
                if(prod in RecommendationDict.keys()):
                    continue
                
                RecommendationDict[prod] = prob
    #return recommendations
    return RecommendationDict

In [ ]:
#getRecommendataionsForCustomer
#  Take a list of orders, a unique customer id in the orders and a dictionary containing Category Groups of the form:
#      key : Unique Group Identifier in Category Group
#      value : dict{
#                     'Groups': DF of Groups (rows) and Categories (columns),
#                     'DropCols': columns to ignore in the DF,
#                     'CategoryColName': column name in orders that contains values used as columns for DF
#                  }
#
#  For each Category Group, find recommendations using cosine simularity and Agglomerative clustering that 
#  are above a minimum threshold provided. 
#
#  Gather the recommendations into a DataFrame and sum them into a column called Total.  Return a DataFrame
#  of recommendations for the user sorted by the total score
#
#  orders - a list of orders from the OL_Data dataset, either the whole thing or simplified columns
#  custUniqueId - the ID of the customer to create recommendations for
#  CatgoryGroupDict - Dictionary of Category Groups, as discribed above
#  minimumSimularity - value between 0 and 1, any simularity lower than this will be ignored
#  includeRFM (optional) - Boolean to include/exclude the RFM score in recommendation
#  RFM_ColName (optional) - Column for RFM Score, only used if includeRFM is True
def getRecommendataionsForCustomer(orders,
                                   customerUniqueId,
                                    CategoryGroupDict,
                                    minimumSimularity,
                                    includeRFM = False):
    #Array to make DF from
    recomendationArray = []
    
    
    for i, (k,v) in enumerate(CategoryGroupDict.items()):
        
        #Add Cosine Simularity Recommendations to the Array
        recomendationArray.append(
            getRecommendataionsFromCategoryGroupDict(
                                    orders,
                                    customerUniqueId,
                                    'product_id',
                                    'customer_unique_id',
                                    v['Groups'],
                                    v['DropCols'],
                                    v['CategoryColName'],
                                    k,
                                    minimumSimularity,
                                    includeRFM,
                                    v['RFMCol'])
        )
        #Add Agglomerative Simularity Recommendations to the Array
        recomendationArray.append(
            getRecommendataionsFromAglomCluster(
                                    orders,
                                    customerUniqueId,
                                    'product_id',
                                    'customer_unique_id',
                                    v['Groups'],
                                    v['DropCols'],
                                    v['CategoryColName'],
                                    k,
                                    minimumSimularity,
                                    includeRFM,
                                    v['RFMCol'])
        )
    
    #Create Product Rec DF from array, and fill NA with 0
    recs = pd.DataFrame(recomendationArray)
    allRecs = recs.rename(columns={'IdType':'index'}).set_index('index').T.reset_index().rename(columns={'index':'product_id'}).fillna(0)
    
    #Create Total Column, a sum of all the other recommendation scores
    scoreTypes = recs.IdType.unique()
    allRecs['Total'] = np.zeros(len(allRecs))
    for scoreType in scoreTypes:
        allRecs['Total'] = allRecs['Total']+allRecs[scoreType]
    
    #Return Recommendations
    return allRecs.sort_values(by='Total', ascending=False)

Import Data

In [ ]:
#Import Order Data
OL_Data = pd.read_csv("./dataset/OL_Data_Cat_Simplified.csv")

OL_Data_RFM = pd.read_csv("./OLData_RFM_Metrics.csv")
buyerRFMDF = OL_Data_RFM[['customer_unique_id','Buyer_OverallRFMScore']].drop_duplicates()
sellerRFMDF = OL_Data_RFM[['seller_id','Seller_OverallRFMScore']].drop_duplicates()

In [ ]:
#Simplify the Orders Data Frame
OL_Data_Simp = OL_Data[['order_id',
                        'customer_unique_id',
                        'product_id',
                        'product_category_name_english',
                        'seller_id',
                        'SellerGroupId',
                        'CustomerGroupId',
                        'CustomerProductGroupId']]
OL_Data_Simp = OL_Data_Simp.drop_duplicates()
OL_Data_Simp = OL_Data_Simp.merge(buyerRFMDF, how='left', on='customer_unique_id') 
OL_Data_Simp = OL_Data_Simp.merge(sellerRFMDF, how='left', on='seller_id') 

In [ ]:
#Generate Category Groups Dict to Generate Recommendations
CategoryGroups = {}

#Add Seller-Category Groups
SellerCategoryGroups = pd.read_csv("./SellerCategoryUniqueGroups.csv").sort_values(by='SellerGroupId')
SellerSimularityScores = pd.read_csv("./SellerCategoryCosineSimularity.csv").sort_values(by='SellerGroupId')
SellerCatDict = {'Groups':SellerCategoryGroups, 
                 'DropCols':['size'],
                 'CategoryColName':'product_category_name_english',
                 'CatIdentifier':'seller_id',
                 'Similarity': SellerSimularityScores,
                 'RFMCol':'Seller_OverallRFMScore'}
CategoryGroups['SellerGroupId']=SellerCatDict

#Add Buyer-Category Groups
CustomerCategoryGroups = pd.read_csv("./BuyerCategoryUniqueGroups.csv").sort_values(by='CustomerGroupId')
CustomerSimularityScores = pd.read_csv("./BuyerCategoryCosineSimularity.csv").sort_values(by='CustomerGroupId')
CustomerCatDict = {'Groups':CustomerCategoryGroups, 
                 'DropCols':['size'],
                 'CategoryColName':'product_category_name_english',
                 'CatIdentifier':'customer_unique_id',
                 'Similarity': CustomerSimularityScores,
                 'RFMCol':'Buyer_OverallRFMScore'}
CategoryGroups['CustomerGroupId']=CustomerCatDict

#Add Customer-Product Groups
CustomerProductGroups = pd.read_csv("./Product_Customer_UniqueGroups.csv").sort_values(by='CustomerProductGroupId')
CustomerProductSimularityScores = pd.read_csv("./Product_Customer_CosineSimularity.csv").sort_values(by='CustomerProductGroupId')
ProductSimDict = {'Groups':CustomerProductGroups, 
                    'DropCols':['size'],
                    'CategoryColName':'product_id',
                    'CatIdentifier':'customer_unique_id',
                    'Similarity': CustomerProductSimularityScores,
                    'RFMCol':'Buyer_OverallRFMScore'}
CategoryGroups['CustomerProductGroupId']=ProductSimDict

# Генерація рекомендацій

Знаходження спільних кластерів айдішок

In [ ]:
OL_Data_Simp.customer_unique_id.value_counts().head(5)

Покупки для випадкового кастомера

In [ ]:
exampleCustomer = '3e43e6105506432c953e165fb2acf44c'
OL_Data_Simp[OL_Data_Simp.customer_unique_id == exampleCustomer].product_category_name_english.value_counts()

## Рекомендація

In [ ]:
#Product Ids with Category
prodCategories = OL_Data_Simp[['product_id','product_category_name_english']].drop_duplicates()

#Standard
recs = getRecommendataionsForCustomer(OL_Data_Simp, exampleCustomer, CategoryGroups, 0.8)
recs = recs.sort_values(by='Total', ascending=False)
recs = recs.merge(prodCategories, how='left', on='product_id')

#With RFM Consideration
recs2 = getRecommendataionsForCustomer(OL_Data_Simp, exampleCustomer, CategoryGroups, 0.8, True)
recs2 = recs2.sort_values(by='Total', ascending=False)
recs2 = recs2.merge(prodCategories, how='left', on='product_id')

In [ ]:
recsSub = recs.head(200)
recsSub.loc[:,recsSub.any()]

In [ ]:
recs.head(200).product_category_name_english.value_counts()

In [ ]:
recs2.head(200).product_category_name_english.value_counts()

Оновлення рекомендації

In [ ]:
#getRecommendataionsFromCategoryGroupDict
#  Take a Category DataFrame (columns represent categories of the Orders, and each row is a Group that sums 
#  to 100% with this percentage spread out among the columns and every row is unique) and a simularity matrix
#  between the Groups of the Category DataFrame, and update them based on new orders 
#
#  This function will update the categories on a customer by customer basis, needing the column to identify
#  unique customers and the old orders the original Category DataFrame and Simularity Matrix was made from
#  to update them.  This is not the most effiecient method, as customers perchase percentages are most likely 
#  repeated, but it does the job.  Improving the performance is left for future work
#
#  categoryGroupsDF - Category DataFrame as described above, columns are categories, rows are Groups
#  categoryGroupsDF_DropCols - Columns in categoryGroupsDF to ignore
#  categoryGroupsDF_CatColumnName - the column name in the orders that contains the categories in the columns
#                                   of categoryGroupsDF
#  categoryGroupsDF_groupIdName - the column name of the unique Group (row) identifier in categoryGroupsDF
#  simularityScores - Matrix representing the simularity between every Group in the Category Groups DF Rows
#  uniqueCustIdCol - name of the column that contains the unique customer identifier
#  oldOrders - Orders the original Category Groups DataFrame and Simularity Matrix were made from
#  newOrders - Orders to add/modify the Category Groups DataFrame and Simularity Matrix
def UpdateCategoryGroupsForNewSales(categoryGroupsDF,
                                    categoryGroupsDF_CatColumnName,
                                    categoryGroupsDF_DropCols,
                                    categoryGroupsDF_groupIdName,
                                    
                                    simularityScores,
                                    uniqueCustIdCol,
                                    oldOrders,
                                    newOrders):
    
    #Gather all orders
    allOrders = pd.concat([oldOrders, newOrders])
    #Boolean to represent whether new Groups have been made
    newGroups = False
    
    #Parameters used for Tracking Progress
    ind = 0
    lengthOfNewUsers = len(newOrders[uniqueCustIdCol].unique())
    print("Updating orders of {0} users".format(lengthOfNewUsers))
    
    #Update Data Structures on a user by user basis
    for custId in newOrders[uniqueCustIdCol].unique():
        #Print update
        if(lengthOfNewUsers>100 and ind%(lengthOfNewUsers//10) ==0):
                print("{0}% ".format(ind/lengthOfNewUsers*100))
        ind= ind+1
        
        #Get old Category Group
        oldCategory = oldOrders[oldOrders[uniqueCustIdCol]==custId][categoryGroupsDF_groupIdName]
        
        #Index Category by Group Id
        catGroupsDF_Indexed = categoryGroupsDF.set_index(categoryGroupsDF_groupIdName).sort_values(by=categoryGroupsDF_groupIdName)
        if(len(oldCategory) != 0):
            #If Old Category exists, decrease count by one
            #This will be replaced if new orders do not alter the percetage distribution of orders
            oldCategory = oldCategory[0]
            catGroupsDF_Indexed.loc[oldCategory,'size'] = catGroupsDF_Indexed.loc[oldCategory,'size'] - 1

        #Remove ignored columns from CategoryGroupsDF
        catGroupsDF_Dropped = catGroupsDF_Indexed.drop(columns=categoryGroupsDF_DropCols)

        #Get all categories in the Group
        all_categories = list(catGroupsDF_Dropped.columns)
        all_categories = [str(x) for x in all_categories]        

        #Make Customer Row From Old Orders
        thiscustOldSubset = oldOrders[oldOrders[uniqueCustIdCol] == custId]
        thisCustOld = np.array([len(thiscustOldSubset[thiscustOldSubset[categoryGroupsDF_CatColumnName] == x]) for x in all_categories])
        #And Customer Row From New Orders
        thiscustNewSubset = newOrders[newOrders[uniqueCustIdCol] == custId]
        thisCustNew = np.array([len(thiscustNewSubset[thiscustNewSubset[categoryGroupsDF_CatColumnName] == x]) for x in all_categories])
        
        #Normalize Customer Row to 100%
        custTotal = thisCustOld+thisCustNew
        custTotalNorm = custTotal/sum(custTotal)*100/5.0
        custTotalNorm = np.floor(custTotalNorm)
        custTotalNorm = custTotalNorm * 5.0
        custTotalNorm.shape=(1,len(custTotalNorm))
        
        #Create Customer DataFrame
        custTotalNormDF = pd.DataFrame(data=custTotalNorm, columns=all_categories)    
        #Get similarity between Customer DataFrame and Existing Groups
        simScores = np.array(cosine_similarity(catGroupsDF_Dropped,custTotalNormDF))
        
        if(max(simScores)>=.9999):
            #Simularity is near enough to an existing Group
            
            #Create and Sort Simularity Scores
            simDF = pd.DataFrame(simScores, columns=['Simularity'])
            simDF[categoryGroupsDF_groupIdName] = catGroupsDF_Dropped.reset_index()[categoryGroupsDF_groupIdName]
            simDF = simDF[simDF.Simularity == max(simDF.Simularity)]
            
            #Find New Category for this Customer, and add 1 to the size of that group
            newCat = simDF.head(1)[categoryGroupsDF_groupIdName]
            catGroupsDF_Indexed.loc[newCat,'size'] = catGroupsDF_Indexed.loc[newCat,'size'] + 1
            
            #Reset CategroyGroup Index and set all the customer's orders to new Category
            categoryGroupsDF = catGroupsDF_Indexed.reset_index()
            allOrders.loc[allOrders[uniqueCustIdCol]==custId,categoryGroupsDF_groupIdName] = newCat
        else:
            #Matching Group Does not Exist, Create a new one
            newGroups = True
            #New Category Numer
            newCat = max(categoryGroupsDF[categoryGroupsDF_groupIdName])+1
            
            #Add Category Number and size to Customer Row DF
            custTotalNormDF[categoryGroupsDF_groupIdName] = newCat
            custTotalNormDF['size'] = 1
            #Add to Category Groups
            categoryGroupsDF = pd.concat([categoryGroupsDF,custTotalNormDF])
            
            #Add simularity Scores to Simularity Matrix
            simCols = simularityScores.set_index(categoryGroupsDF_groupIdName).columns
            length = len(simScores)
            simScores.shape = (1, length)
            
            #New Group Simularity DF Row
            custSimScoresDF = pd.DataFrame(data=simScores, columns=simCols)
            custSimScoresDF[categoryGroupsDF_groupIdName] = newCat
            custSimScoresDF[str(newCat)] = 1.0
            
            #Add new column for new scores
            simScores.shape = (length,1)
            simularityScores[str(newCat)] = simScores
            #Add New Group Row to Sim Scores
            simularityScores = pd.concat([simularityScores,custSimScoresDF])
            
            #Update all customer orders
            allOrders.loc[allOrders[uniqueCustIdCol]==custId,categoryGroupsDF_groupIdName] = newCat
        
    
    if(all(categoryGroupsDF['size'] != 0) and not newGroups):
        #If there are no groups added or removed, no need to go any further
        return (allOrders, categoryGroupsDF, simularityScores)
    
    #Set Indexes
    catGroupsDF_Indexed = categoryGroupsDF.set_index(categoryGroupsDF_groupIdName)
    simScore_Indexed = simularityScores.set_index(categoryGroupsDF_groupIdName)
    
    if(any(categoryGroupsDF['size'] <= 0)):
        #If some groups are now empty, remove them from Category Groups and Simularity Scores
        groupsToDrop = categoryGroupsDF[categoryGroupsDF.size == 0][categoryGroupsDF_groupIdName]
    
        for group in groupsToDrop:
            catGroupsDF_Indexed = catGroupsDF_Indexed.drop(group)
            simScore_Indexed = simScore_Indexed.drop(group)
            simScore_Indexed = simScore_Indexed.drop(columns=group)
    
    #Reset Indexes
    categoryGroupsDF = catGroupsDF_Indexed.reset_index()
    simularityScores = simScore_Indexed.reset_index()
    
    #Renumber all Groups, so that they are of a continuous range
    #creating dictionary from old group number to new
    groupsIndOld = categoryGroupsDF[categoryGroupsDF_groupIdName]
    groupsIndNew = list(range(len(groupsIndOld)))
    changeDict = {groupsIndOld[i] : groupsIndNew[i] for i in range(len(groupsIndOld))}
    
    #Update Orders to new Group Numbers
    allOrders.replace({categoryGroupsDF_groupIdName:changeDict})
    #Update Category Groups to new Group Numbers
    categoryGroupsDF[categoryGroupsDF_groupIdName] = groupsIndNew
    categoryGroupsDF = categoryGroupsDF.rename(columns=changeDict)
    #Update Simularity Scores to new Group Numbers
    simularityScores[categoryGroupsDF_groupIdName] = groupsIndNew
    simularityScores = simularityScores.rename(columns=changeDict)
    
    return (allOrders, categoryGroupsDF, simularityScores)

# Приклад використання

In [ ]:
#Create Subset of groups, representing the first 5 Customer Groups
custGroups = range(0,5)
CustomerCategoryGroupsSub = CustomerCategoryGroups[CustomerCategoryGroups.CustomerGroupId.isin(custGroups)]
#Subset Of Simularity Scores
CustomerSimularityScoresSub = CustomerSimularityScores[CustomerSimularityScores.CustomerGroupId.isin(custGroups)]
columnsSubset = ['CustomerGroupId'] + [str(x) for x in custGroups]
CustomerSimularityScoresSub = CustomerSimularityScoresSub[columnsSubset]
#Subset of Orders
OrdersSub = OL_Data_Simp[OL_Data_Simp.CustomerGroupId.isin(custGroups)]

#New Orders Subset
newCustGrous = range(100,105)
newOrders = OL_Data_Simp[OL_Data_Simp.CustomerGroupId.isin(newCustGrous)]

In [ ]:
#Old Category Groups
CustomerCategoryGroupsSub.loc[:, CustomerCategoryGroupsSub.any()]

In [ ]:
#Old Simularity Scores
CustomerSimularityScoresSub

In [ ]:
CategoryGroup = CategoryGroups['CustomerGroupId']
(orders, cat, sim) = UpdateCategoryGroupsForNewSales(CustomerCategoryGroupsSub,
                                                        CategoryGroup['CategoryColName'],
                                                        CategoryGroup['DropCols'],
                                                        'CustomerGroupId',
                                                        CustomerSimularityScoresSub,
                                                        'customer_unique_id',
                                                        OrdersSub,
                                                        newOrders)

In [ ]:
#New Categories
cat.loc[:,cat.any()]

In [ ]:
#New Simularity
sim